In [ ]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

# 11장 시계열(time series)
* 금융, 경제, 생태학, 신경 과학, 서버 로그 데이터 분석 등 분야에서 사용되는 구조화된 데이터
* 시간상의 여러 지점을 관측하거나 측정할수 있는 모든 것
* 시간 내에서 특정 순간의 타임스탬프로 색인된 데이터

## 1. datetime 모듈의 자료형
* datetime: 날짜와 시간을 함께 나타내는 자료형
* date: 날짜(연, 월, 일)만을 나타내는 자료형
* time: 시간만을 나타내는 자료형
* timedelta: 두 날짜 또는 시간 사이의 차이를 나타내는 자료형



## 2. datetime의 속성
* year: 년도
* month: 월
* day: 일
* hour: 시간
* minute: 분
* second: 초
* microsecond: 마이크로초

## 3. datetime 자료형의 메서드
* now(): 현재 날짜와 시간을 반환
* today(): 오늘 날짜를 반환

In [ ]:
from datetime import datetime
now = datetime.now()
print(now)
print(type(now))

In [ ]:
now.year
now.month
now.day

## 4. timedelta 자료형의 속성
* days: 일
* seconds: 초
* microseconds: 마이크로초

In [ ]:
delta =  datetime.now() - datetime(2024, 1, 1)
delta.days

## 5. datetime 포맷 규칙
* %Y: 년도 (4자리)
* %m: 월 (2자리)
* %d: 일 (2자리)
* %H: 시간 (24시간제, 2자리)
* %M: 분 (2자리)
* %S: 초 (2자리)


In [ ]:
# datetime -> 문자열 datetime 객체로 변환
stamp = datetime(2024, 5, 6)

str(stamp)

stamp.strftime("%Y-%m-%d")

## 6. 시계열의 종류
* 파이썬 문자열
* datetime 객체로 표현되는 타임스템프로 색인된 Series

In [ ]:
# 문자열 -> datetime 날짜로 파싱
value = "2011-01-03"
datetime.strptime(value, "%Y-%m-%d")


In [ ]:
# 날짜 문자열 리스트 -> datetime으로 파싱
datestrs = ["2011-07-06 12:00:00", "2011-08-06 00:00:00"]
pd.to_datetime(datestrs)


In [ ]:
idx = pd.to_datetime(datestrs + [None])
idx
idx[2]
pd.isna(idx)

## 7. 시계열의 색인, 선택, 부분 선택


In [ ]:
# (1) datetime 타임스템프로 색인된 Series 객체
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.standard_normal(6), index=dates)
ts

In [ ]:
ts.index # DatetimeIndex

In [ ]:
stamp = ts.index[0]
ts[stamp]

In [ ]:
ts["2011-01-10"]

In [ ]:
ts[::2]


### [실습] 가상 시계열 데이터 생성
* pandas.date_range()  
  * pandas 라이브러리의 함수로, 특정 기간 내의 날짜 범위를 생성
  * start: 시작 날짜
  * end: 종료 날짜
  * periods: 생성할 날짜 수
  * freq: 날짜 간격 (기본값: 'D' (하루))

In [ ]:
index = pd.date_range("2012-04-01", "2012-06-01")
pd.date_range(start="2012-04-01", periods=20)
pd.date_range(end="2012-06-01", periods=20)

In [ ]:
# ex>  date_range()함수를 이용하여 날짜 시컨스 생성
longer_ts = pd.Series(np.random.standard_normal(1000),
                      index=pd.date_range("2000-01-01", periods=1000))
longer_ts
longer_ts["2001"]

In [ ]:
longer_ts["2001-05"]

In [ ]:
# ex> 시계열 데이타프레임 생성
dates = pd.date_range("2000-01-01", periods=10, freq="W-WED")
long_df = pd.DataFrame(np.random.standard_normal((10, 4)),
                       index=dates,
                       columns=["Colorado", "Texas",
                                "New York", "Ohio"])
long_df
long_df.loc["2000-01"]

In [ ]:
# grouping
grouped = long_df.groupby(level=0)
for n, g in grouped:
  print(f"name : {n}")
  print(g)
  print()


In [ ]:
grouped.mean()
grouped.count()
grouped.size()

In [ ]:
# ex > 시계열 데이터셋 파일 -> 시계열 색인을 갖는 데이터프레임 객체로 변환
close_px_all = pd.read_csv("examples/stock_px.csv", parse_dates=True, index_col= 0)
close_px_all.head()

In [ ]:
close_px_all.columns  # AAPL: Apple Inc, MSFT: Microsoft Corporation, XOM: Exxon Mobil Corporation, SPX: S&P 500 Index


In [ ]:
close_px = close_px_all[["AAPL", "MSFT", "XOM"]]
close_px


In [ ]:
# (1) line plots
# close_px["AAPL"].plot.bar()

close_px["AAPL"].plot(kind='line')

### < pandas.rolling() 메서드>
* pandas 라이브러리의 함수로, 특정 기간 동안의 데이터를 그룹화하여 통계를 계산
* 롤링창은 시계열 데이터 분석에 매우 중요
* 롤링창을 사용하여 데이터 변화 추세, 변동성 측정, 이상치 감지, 예측에 사용
* 롤링창을 사용하여 롤링창 내의 데이터 평균, 표준편차, 최대값/최소값, 회귀분석 등
* rolling() 함수의 인자:
  * window: 그룹화할 데이터의 수
  * min_periods: 계산에 포함할 최소 데이터 수 (기본값: window)
  * center: 그룹화할 데이터의 중심을 맞출지 여부 (기본값: False)
  * win_type: 그룹화할 데이터의 가중치 (기본값: None)


In [ ]:
# (1)  롤링창 만들어 통계 계산
# DataFrame의 "AAPL" 열에 대한 rolling() 메서드를 사용하여 크기가 250인 롤링 창을 만드는 방법:
# 롤링 창 사용하기: 롤링 창을 만들면 다양한 통계 함수를 사용하여 창 내의 데이터를 분석
plt.figure()
print(close_px["AAPL"].rolling(250).mean())
close_px["AAPL"].rolling(250).mean().plot()


In [ ]:
# (2) 롤링창 만들어 통계 계산 - 2 개의 종가에 대한 250일 롤링 평균 계산
plt.figure()
close_px["AAPL"].rolling(window = 250).mean().plot()
close_px["XOM"].rolling(window = 250).mean().plot()



In [ ]:
# (3) 롤링창 만들어 통계 계산 - 모든 종목의 종가에 대한 60일 롤링 평균 계산:
# 2. 60일 롤링 평균을 로그 스케일로 플롯: y축이 로그 스케일로 플롯
plt.figure()
close_px.rolling(60).mean().plot(logy=True)


In [ ]:
# # Create a figure
plt.figure()
# Select the desired date range for AAPL stock prices
aapl_px = close_px["AAPL"]["2006":"2007"]
# Calculate the 30-day simple moving average
ma30 = aapl_px.rolling(30, min_periods=20).mean()
# Calculate the 30-day exponential moving average
ewma30 = aapl_px.ewm(span=30).mean()
# Plot the AAPL stock prices : sets the line style to solid (-) and the color to blue (b).
aapl_px.plot(style="b-", label="Price")
# Plot the 30-day simple moving average
ma30.plot(style="k--", label="Simple Moving Avg")
# Plot the 30-day exponential moving average
ewma30.plot(style="k-", label="EW MA")
plt.legend()
plt.show()